<a href="https://colab.research.google.com/github/SunbalAzizLCWU/BSSE-DS-Project/blob/main/SunbalW10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import shutil
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [4]:
print("--- Step 1: Cleaning Environment ---")

# 1. Check for kaggle.json
if not os.path.exists('kaggle.json'):
    print("❌ ERROR: 'kaggle.json' is missing.")
    print("Please upload it to the Files tab on the left!")
    raise SystemExit("Stop: Missing kaggle.json")

# 2. Setup Kaggle Config
if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')
os.system('cp kaggle.json /root/.kaggle/')
os.system('chmod 600 /root/.kaggle/kaggle.json')

# 3. Force Clean Old Data
if os.path.exists('Garbage classification'):
    shutil.rmtree('Garbage classification')
    print("Deleted old data folder to ensure a fresh start.")

--- Step 1: Cleaning Environment ---


In [5]:
print("\n--- Step 2: Downloading Dataset ---")
os.system('kaggle datasets download -d asdasdasasdas/garbage-classification')
os.system('unzip -q garbage-classification.zip')
print("Dataset downloaded and unzipped.")


--- Step 2: Downloading Dataset ---
Dataset downloaded and unzipped.


In [6]:
print("\n--- Step 3: Finding and Loading Images (CNN Mode) ---")
CLASSES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
IMG_SIZE = 64


--- Step 3: Finding and Loading Images (CNN Mode) ---


In [7]:
# AUTOMATIC PATH FINDER
# We check all possible locations where the data might hide
possible_paths = [
    'Garbage classification/Garbage classification',
    'Garbage classification',
    './Garbage classification/Garbage classification'
]

DATA_DIR = None
for path in possible_paths:
    # Check if this path contains the 'cardboard' folder
    if os.path.exists(os.path.join(path, 'cardboard')):
        DATA_DIR = path
        print(f"✅ Found dataset at: {DATA_DIR}")
        break

if DATA_DIR is None:
    # Debug: List what IS there
    print("❌ Could not find data folders. Current files:")
    print(os.listdir('.'))
    if os.path.exists('Garbage classification'):
        print("Inside 'Garbage classification':", os.listdir('Garbage classification'))
    raise SystemExit("Stop: Folder structure unknown.")

X_cnn_data = []
y_labels = []

for class_name in CLASSES:
    class_dir = os.path.join(DATA_DIR, class_name)
    if not os.path.exists(class_dir):
        continue

    files = os.listdir(class_dir)
    # Loop through files
    for image_file in files:
        try:
            image_path = os.path.join(class_dir, image_file)
            img = Image.open(image_path).convert('L') # Grayscale
            img_resized = img.resize((IMG_SIZE, IMG_SIZE))

            # KEY DIFFERENCE FOR WEEK 10: DO NOT FLATTEN
            # We keep it as a 2D matrix (64, 64)
            img_array = np.array(img_resized)

            X_cnn_data.append(img_array)
            y_labels.append(CLASSES.index(class_name))
        except Exception:
            pass

X = np.array(X_cnn_data)
y = np.array(y_labels)

print(f"✅ Loaded {len(X)} images. Shape: {X.shape}")

✅ Found dataset at: Garbage classification/Garbage classification
✅ Loaded 2527 images. Shape: (2527, 64, 64)


In [8]:
if len(X) == 0:
    raise SystemExit("Stop: No images loaded.")

# Reshape for CNN input: (Batch, Height, Width, Channels)
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# Normalize (0-1)
X = X / 255.0

print(f"Data Reshaped for CNN: {X.shape}")

print("\n--- Step 4: Training Week 10 CNN ---")

Data Reshaped for CNN: (2527, 64, 64, 1)

--- Step 4: Training Week 10 CNN ---


In [9]:
# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [10]:
# Build CNN Model
model_cnn = Sequential([
    # Convolution 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),

    # Convolution 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Convolution 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Flatten (ONLY at the end)
    Flatten(),

    # Dense Layers
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(6, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Train
history = model_cnn.fit(X_train, y_train, validation_data=(X_test, y_test),
                        epochs=40, batch_size=32, callbacks=[early_stop], verbose=1)

# Result
loss, acc_cnn = model_cnn.evaluate(X_test, y_test)
print(f"\n==========================================")
print(f"🎉 Week 10 Result: CNN Test Accuracy: {acc_cnn*100:.2f}%")
print(f"==========================================")

Epoch 1/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 22s 304ms/step - accuracy: 0.2115 - loss: 1.7379 - val_accuracy: 0.3221 - val_loss: 1.6799
Epoch 2/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 17s 264ms/step - accuracy: 0.2877 - loss: 1.6399 - val_accuracy: 0.3498 - val_loss: 1.5156
Epoch 3/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 16s 245ms/step - accuracy: 0.3484 - loss: 1.5583 - val_accuracy: 0.3676 - val_loss: 1.5041
Epoch 4/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 15s 230ms/step - accuracy: 0.3779 - loss: 1.4898 - val_accuracy: 0.4506 - val_loss: 1.4217
Epoch 5/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 16s 255ms/step - accuracy: 0.4111 - loss: 1.4533 - val_accuracy: 0.4664 - val_loss: 1.3327
Epoch 6/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 15s 234ms/step - accuracy: 0.4735 - loss: 1.3516 - val_accuracy: 0.5079 - val_loss: 1.3103
Epoch 7/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 15s 233ms/step - accuracy: 0.4931 - loss: 1.2992 - val_accuracy: 0.5731 - val_loss: 1.2350
Epoch 8/40
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 231ms/step - accuracy: 0.5295 - loss: 1.2163 - val_accu